In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [1]:
def starting_bar_(axis, df, variable_x, variable_y, color, label, size_text):
    """
    Helper function to set the bottom of the bar chart and to print labels of metric within each bar
    """

    axis.bar(list(df[variable_x])
            , list(df[variable_y])
            , width=0.8
            , color=color
            , label = label
           )
    ax_x = [(patch.get_x(), patch.get_y()) for patch in axis.patches]

    ax_height = [(patch.get_x(), patch.get_height()) for patch in axis.patches]


    for x, height in zip(ax_x, ax_height):
        
        if height[1] > 0:
            ax.text(x[0] + 0.3, x[1] +0.8, label
                    , fontsize=size_text
                    , color='white'
                    , rotation = 90
                    , fontweight='bold'
                     , bbox=dict(boxstyle="square"
                                 , color='grey'
                                 , alpha=0.5
                                )
                   )
    return ax_height

In [2]:
def bar_with_text_(axis, df, variable_x, variable_y, bottom, color, label, prev_set,size_text):
    """
    Helper function to stack every additional mteric on top of the previous one and
    to print label of each metric within each new stacked bar
    """

    axis.bar(list(df[variable_x])
            ,list(df[variable_y])
            , bottom=bottom
            , width=0.8
            , color=color
            , label = label
           )



    ax_height = [ patch.get_height() for patch in axis.patches]
    ax_x_values = [ patch.get_x() for patch in axis.patches]

    df_x = (pd.DataFrame(zip(ax_height, ax_x_values)
                          , columns=['height', 'x_values']).groupby('x_values').height.sum().reset_index()
            )

    set_xy = set([(x, y) for x,y in zip(df_x.x_values, df_x.height)])

    coords = list(set_xy - set(prev_set))
    

    for x in coords:
            axis.text(x[0] + 0.3, x[1] - 0.1 
                      , label
                      , fontsize=size_text
                      , color='white'
                      , rotation = 90
                      , fontweight='bold'
                      , bbox=dict(boxstyle="square"
                                 , color='grey'
                                 , alpha=0.5
                        )
                     )
    return set_xy

In [4]:
# fig = plt.figure(figsize=(20,17))
# ax = fig.add_subplot(111)
#Has fig and axis to be defined previously 

def make_plot(df, x_variable,list_y, list_colors, size_text):
    """
    Stacks each binary (1/0) flag on top of each other for a given x and prints its label in the bar
    Example Use Csse:
    Stacks flags for specific days on top of each other in order to see which day is the most flagged
    IN: 
    Dataframe 
    x_variable: Some dimension (ints)
    list_y: List of flags (binary_flag_one, binary_flag_two, etc)
    list_colors: list of colors like ['r', 'g', 'b'] same length as list_y
    size_text: INT for the size of the text in each stackes bar
    Out:
    plot where flags are stacked on top of each other and their label is printed within each stacked bar
    """

    
    from copy import deepcopy
    
    list_to_pop = deepcopy(list_y)
    list_color_pop = deepcopy(list_colors)
    
    variable_y_1 = list_to_pop.pop()
    
    prev_set=\
    (starting_bar_(axis=ax
                 , df=df
                 , variable_x=x_variable
                 , variable_y= variable_y_1
                 , color= list_color_pop.pop()
                 , label= variable_y_1
                  , size_text = size_text
                )
    )
    
    bottom_new=np.array(df[variable_y_1])

    count = len(list_y) - 1
    
    while count > 0:
        
        variable_y_2 = list_to_pop.pop()
        prev_set=\
        (bar_with_text_(axis=ax
                       , df=df
                       , variable_x=x_variable
                       , variable_y = variable_y_2
                       , bottom=bottom_new
                       , color=list_color_pop.pop()
                       , label=variable_y_2
                       , prev_set=prev_set
                       , size_text = size_text
                      )
        )
        
        bottom_new = bottom_new + np.array(df[variable_y_2])
    
        count -= 1
    
    return bottom_new